In [ ]:
# Clone repository
%cd /content
!rm -rf Deep_Learning_Project_Gil_Alon
!git clone https://github.com/gil-attar/Deep_Learning_Project_Gil_Alon.git
%cd Deep_Learning_Project_Gil_Alon

In [ ]:
# Install dependencies
!pip install ultralytics roboflow pyyaml -q

In [ ]:
# Verify GPU is available
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Download Dataset

In [ ]:
import os

# Set Roboflow API key
os.environ["ROBOFLOW_API_KEY"] = "zEF9icmDY2oTcPkaDcQY"

# Download dataset
!python scripts/download_dataset.py --output_dir data/raw

In [ ]:
# Verify dataset
!echo "Train: $(ls data/raw/train/images/ | wc -l) images"
!echo "Valid: $(ls data/raw/valid/images/ | wc -l) images"
!echo "Test: $(ls data/raw/test/images/ | wc -l) images"

## 3. Prepare Training Configuration

Create a data.yaml with correct paths for Colab.

In [ ]:
import yaml
from pathlib import Path

# Load original data.yaml to get class names
with open('data/raw/data.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Create training config with absolute paths
train_config = {
    'path': '/content/Deep_Learning_Project_Gil_Alon/data/raw',
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'names': config['names'],
    'nc': len(config['names'])
}

# Save training config
with open('data/train_config.yaml', 'w') as f:
    yaml.dump(train_config, f, default_flow_style=False)

print("Training config saved to data/train_config.yaml")
print(f"Number of classes: {train_config['nc']}")
print(f"Classes: {train_config['names'][:5]}..." if len(train_config['names']) > 5 else f"Classes: {train_config['names']}")

## 4. Train YOLOv8 (CNN Baseline)

Using YOLOv8n (nano) - smallest and fastest variant.

Training parameters:
- **Epochs:** 50 (can adjust based on convergence)
- **Image size:** 640x640
- **Batch size:** Auto (based on GPU memory)

In [ ]:
from ultralytics import YOLO

# Load pretrained YOLOv8n
yolo_model = YOLO('yolov8n.pt')

# Train on our dataset
yolo_results = yolo_model.train(
    data='data/train_config.yaml',
    epochs=50,
    imgsz=640,
    batch=-1,  # Auto batch size
    patience=10,  # Early stopping
    save=True,
    project='runs/train',
    name='yolov8n_baseline',
    exist_ok=True,
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=42
)

In [ ]:
# Save best weights to models folder
import shutil
from pathlib import Path

Path('models').mkdir(exist_ok=True)

# Try multiple possible locations (Ultralytics may save in different paths)
possible_paths = [
    'runs/train/yolov8n_baseline/weights/best.pt',
    'runs/detect/train/yolov8n_baseline/weights/best.pt',
    'runs/detect/runs/train/yolov8n_baseline/weights/best.pt'
]

yolo_best = None
for path in possible_paths:
    p = Path(path)
    if p.exists():
        yolo_best = p
        print(f"Found weights at: {path}")
        break

if yolo_best:
    shutil.copy(yolo_best, 'models/yolov8n_baseline.pt')
    print("✓ YOLOv8n weights saved to models/yolov8n_baseline.pt")
else:
    print("❌ Warning: best.pt not found in any expected location")
    print("Run this to find it: !find runs -name 'best.pt' -path '*/yolov8n_baseline/*'")

## 5. Train RT-DETR (Transformer Baseline)

Using RT-DETR-l (large) - best balance of speed and accuracy.

Same training parameters as YOLOv8 for fair comparison.

In [ ]:
from ultralytics import RTDETR

# Load pretrained RT-DETR-l
rtdetr_model = RTDETR('rtdetr-l.pt')

# Train on our dataset
rtdetr_results = rtdetr_model.train(
    data='data/train_config.yaml',
    epochs=50,
    imgsz=640,
    batch=-1,  # Auto batch size
    patience=10,  # Early stopping
    save=True,
    project='runs/train',
    name='rtdetr_baseline',
    exist_ok=True,
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=42
)

In [ ]:
# Save best weights
possible_paths = [
    'runs/train/rtdetr_baseline/weights/best.pt',
    'runs/detect/train/rtdetr_baseline/weights/best.pt',
    'runs/detect/runs/train/rtdetr_baseline/weights/best.pt'
]

rtdetr_best = None
for path in possible_paths:
    p = Path(path)
    if p.exists():
        rtdetr_best = p
        print(f"Found weights at: {path}")
        break

if rtdetr_best:
    shutil.copy(rtdetr_best, 'models/rtdetr_baseline.pt')
    print("✓ RT-DETR weights saved to models/rtdetr_baseline.pt")
else:
    print("❌ Warning: best.pt not found in any expected location")
    print("Run this to find it: !find runs -name 'best.pt' -path '*/rtdetr_baseline/*'")

## 6. Quick Validation

Run inference on a few test images to verify models work.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# Get 3 test images
test_images = list(Path('data/raw/test/images').glob('*.jpg'))[:3]

# Load trained models
yolo_trained = YOLO('models/yolov8n_baseline.pt')
rtdetr_trained = RTDETR('models/rtdetr_baseline.pt')

fig, axes = plt.subplots(3, 3, figsize=(15, 15))

for i, img_path in enumerate(test_images):
    # Original image
    img = Image.open(img_path)
    axes[i, 0].imshow(img)
    axes[i, 0].set_title('Original')
    axes[i, 0].axis('off')
    
    # YOLOv8 prediction
    yolo_result = yolo_trained.predict(img_path, verbose=False)[0]
    yolo_img = yolo_result.plot()
    axes[i, 1].imshow(yolo_img)
    axes[i, 1].set_title(f'YOLOv8 ({len(yolo_result.boxes)} detections)')
    axes[i, 1].axis('off')
    
    # RT-DETR prediction
    rtdetr_result = rtdetr_trained.predict(img_path, verbose=False)[0]
    rtdetr_img = rtdetr_result.plot()
    axes[i, 2].imshow(rtdetr_img)
    axes[i, 2].set_title(f'RT-DETR ({len(rtdetr_result.boxes)} detections)')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.savefig('evaluation/baseline_comparison.png', dpi=150)
plt.show()
print("\n✓ Saved comparison to evaluation/baseline_comparison.png")

## 7. Training Summary

In [ ]:
print("=" * 60)
print("STEP 3 COMPLETE: Baseline Model Training")
print("=" * 60)

print("\nModels trained:")
print(f"  ✓ YOLOv8n: models/yolov8n_baseline.pt")
print(f"  ✓ RT-DETR-l: models/rtdetr_baseline.pt")

print("\nTraining logs:")
print(f"  - runs/train/yolov8n_baseline/")
print(f"  - runs/train/rtdetr_baseline/")

print("\nNext steps:")
print("  1. Download model weights from models/ folder")
print("  2. Step 4: Run evaluation on test set")
print("  3. Step 5: Occlusion difficulty analysis")

## 9. Download Weights & JSON Files

Download the trained model weights and evaluation JSONs to commit to GitHub.

## 8. Step 3.2 - Baseline Evaluation (Generate JSONs)

Run evaluation script to generate all 6 required JSON files for your friend's analysis.

In [ ]:
# First, create data.yaml in processed/ directory
!python scripts/create_data_yaml.py --dataset_root data/raw --output data/processed/data.yaml --absolute

In [ ]:
# Run baseline evaluation for both models
# This generates all 6 JSON files required for Step 3.2

!python scripts/evaluate_baseline.py \
    --yolo_weights models/yolov8n_baseline.pt \
    --rtdetr_weights models/rtdetr_baseline.pt \
    --dataset_root data \
    --output_dir evaluation/metrics \
    --conf_threshold 0.25 \
    --imgsz 640 \
    --model both

In [ ]:
# Verify all JSON files were created
import os

required_files = [
    "evaluation/metrics/baseline_yolo_run.json",
    "evaluation/metrics/baseline_rtdetr_run.json",
    "evaluation/metrics/baseline_yolo_metrics.json",
    "evaluation/metrics/baseline_rtdetr_metrics.json",
    "evaluation/metrics/baseline_yolo_predictions.json",
    "evaluation/metrics/baseline_rtdetr_predictions.json"
]

print("Checking required JSON files:")
print("="*60)
for file_path in required_files:
    exists = os.path.exists(file_path)
    status = "✓" if exists else "✗"
    size = os.path.getsize(file_path) if exists else 0
    print(f"{status} {file_path} ({size/1024:.1f} KB)")

all_exist = all(os.path.exists(f) for f in required_files)
if all_exist:
    print("\n✓ All 6 JSON files generated successfully!")
else:
    print("\n✗ Some JSON files are missing")

In [ ]:
# Display metrics comparison
import json

print("\n" + "="*60)
print("BASELINE METRICS COMPARISON")
print("="*60)

with open("evaluation/metrics/baseline_yolo_metrics.json") as f:
    yolo_metrics = json.load(f)

with open("evaluation/metrics/baseline_rtdetr_metrics.json") as f:
    rtdetr_metrics = json.load(f)

print(f"\n{'Metric':<20} {'YOLOv8n':<15} {'RT-DETR-l':<15} {'Difference':<15}")
print("-"*65)

metrics_to_compare = ["map50", "map50_95", "precision", "recall", "fps"]
for metric in metrics_to_compare:
    yolo_val = yolo_metrics["metrics"][metric]
    rtdetr_val = rtdetr_metrics["metrics"][metric]
    diff = rtdetr_val - yolo_val

    print(f"{metric:<20} {yolo_val:<15.4f} {rtdetr_val:<15.4f} {diff:+.4f}")

print("\n" + "="*60)

In [ ]:
# Save to Google Drive (so you don't lose weights!)
from google.colab import drive
drive.mount('/content/drive')

# Create backup folder
!mkdir -p /content/drive/MyDrive/Deep_Learning_Project/models
!mkdir -p /content/drive/MyDrive/Deep_Learning_Project/evaluation_jsons

# Copy model weights to Drive
!cp models/*.pt /content/drive/MyDrive/Deep_Learning_Project/models/
print("✓ Models saved to Google Drive!")

# Copy JSON files to Drive
!cp evaluation/metrics/*.json /content/drive/MyDrive/Deep_Learning_Project/evaluation_jsons/
print("✓ JSON files saved to Google Drive!")

# Also create zip for manual download
!ls -lh models/
!ls -lh evaluation/metrics/
!zip -r models_and_jsons.zip models/ evaluation/metrics/
print("\n✓ Also download models_and_jsons.zip from the file browser (📁 icon)")

In [ ]:
# Show training results
import pandas as pd
from pathlib import Path

# Try multiple possible locations for YOLOv8 results
yolo_csv_paths = [
    'runs/train/yolov8n_baseline/results.csv',
    'runs/detect/train/yolov8n_baseline/results.csv',
    'runs/detect/runs/train/yolov8n_baseline/results.csv'
]

print("=== YOLOv8 Results ===")
yolo_csv = None
for path in yolo_csv_paths:
    if Path(path).exists():
        yolo_csv = path
        print(f"Found results at: {path}")
        break

if yolo_csv:
    yolo_results = pd.read_csv(yolo_csv)
    print(yolo_results[['metrics/mAP50(B)', 'metrics/mAP50-95(B)']].tail(1))
else:
    print("❌ results.csv not found")

# Try multiple possible locations for RT-DETR results
rtdetr_csv_paths = [
    'runs/train/rtdetr_baseline/results.csv',
    'runs/detect/train/rtdetr_baseline/results.csv',
    'runs/detect/runs/train/rtdetr_baseline/results.csv'
]

print("\n=== RT-DETR Results ===")
rtdetr_csv = None
for path in rtdetr_csv_paths:
    if Path(path).exists():
        rtdetr_csv = path
        print(f"Found results at: {path}")
        break

if rtdetr_csv:
    rtdetr_results = pd.read_csv(rtdetr_csv)
    print(rtdetr_results[['metrics/mAP50(B)', 'metrics/mAP50-95(B)']].tail(1))
else:
    print("❌ results.csv not found")